In [ ]:
!pip install geemap
import geemap, ee
from google.colab import drive
drive.mount('/content/drive')
ee.Authenticate()
ee.Initialize()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 35.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.5/130.5 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 61.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 47.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 40.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 21.6 MB/s eta 0:00:00
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11124 sha256=f3d65261f81f6dd350f9ff2e751c0b8db0204fbbc2c3490bcede3f

In [ ]:
# get our Sidi Bel Abbesl boundary
aoi = ee.FeatureCollection("FAO/GAUL/2015/level1").filter(ee.Filter.eq('ADM1_NAME',"Sidi Bel Abbes")).geometry()

In [ ]:
landsat = ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA')\
            .filterBounds(aoi)\
            .filterDate('2021-01-01', '2021-01-30')
x=landsat.median()



In [ ]:
def getLST(image):
  
    #B=image.select('B10').rename("LST1")
   #B11=image.select('B11')
   # Tb=image.select('B10'),
   # emissivity=image.select('B11')*0.01
   # radiance = image.expression('(0.0003342 * B10) + 0.1') #radiance
   
    lst = image.expression('Tb+(1+(0.00115*(Tb/1.438)*log(emissivity))-273.15)', {
        'Tb': image.select('B10'),
        'emissivity': image.select('B11')
         })
    #lst = radiance.expression('(Tb / (1 + (0.00115 * (Tb / 1.438)) * log(emissivity))) - 273.15') #lst en degree c
    #lst = lst.rename('LST')"""
    lst = lst.rename("LST")
    image = image.addBands(lst)
    
    return (image)

In [ ]:
LST1=getLST(x)
def getNDVI(image):
    
    # Normalized difference vegetation index (NDVI)
    ndvi = image.normalizedDifference(['B5','B4']).rename("NDVI")
    image = image.addBands(ndvi)

    return(image)
a=landsat.map(getNDVI).median()

In [ ]:

color = ['FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718',
               '74A901', '66A000', '529400', '3E8601', '207401', '056201',
               '004C00', '023B01', '012E01', '011D01', '011301']
pallete = {"min":0, "max":60, 'palette':color}
pallet = {"min":0, "max":1, 'palette':color}
b=LST1.select("LST")
map1 = geemap.Map()
map1.centerObject(aoi, 8)
map1.addLayer(a.clip(aoi).select('NDVI'), pallet, "NDVI")
map1.addLayer(b.clip(aoi),pallete, "LST")

map1.addLayerControl()
map1

Map(center=[34.70841918557641, -0.5073845667977872], controls=(WidgetControl(options=['position', 'transparent…

In [ ]:
min = ee.Number(LST1.reduceRegion(ee.Reducer.min(),30).get())
print(min, 'minLST')

EEException: ignored